In [ ]:
### NOTE: Vì đây chỉ là project cá nhân mình làm trong lúc rảnh nên sẽ không tránh được lỗi, và có thể file này chỉ hoạt động ở một số trường đại học như Văn Lang, chúc mọi người thành công :)

### install libary needed
%pip install python-csv
%pip install pandas
%pip install tabula-py
%pip install ipykernel

## import 
import csv
import datetime
import pandas as pd
from tabula.io import convert_into

def merge_relate_row(path):
    ### this will merge relative rows 
    with open(path) as f:
        reader = csv.reader(f)
        lines = list(reader)
        if lines[0][0] == "STT":
            None
        else:
            cur_year = lines[0]
            ###  cur_year lấy năm học hiện tại trong hàng năm học - học kỳ để sau này tính được ngày hiện tại

            lines = lines[1:]
            ### (lines) là các môn học cần sử lý
        data = []
        i = 0
        merged = lines[i].copy()
        if  not lines[i+1][0].isnumeric():
            # nếu như cột đầu trong hàng(merged) là một cột trống thì có thể hàng đó(merged) đó  bị tách ra từ hàng(row) của phía trên  sau đó  thêm vào DATA
            for k in range(0, len(lines[i])-1):
                if merged[k] != '':
                    merged[k] += " "+ lines[i+1][k]
                else:
                    merged[k] +=  lines[i+1][k]
            i += 1
        
        #vì khi add các row vào merged có thể xảy ra trường hợp các hàng dính xuống dòng (enter hoặc \n) nên cần loại bỏ 
        merged=[sub.replace('\n', ' ').strip() for sub in merged]
        data.append(merged)

        i += 1

        while i < len(lines)-1:
            merged = lines[i].copy()
            j = i + 1
            merged = [sub.replace('\n', ' ') for sub in merged]
                    
            i = j
            if merged[0] == "":
                # nếu như cột đầu trong hàng(merged) là một cột trống thì có thể hàng đó(merged) đó  bị tách ra từ hàng(row) của môn phía trên sau đó  thêm vào DATA

                for k in range(0, len(lines[i])-1):

                    data[-1][k] += f" {merged[k]}"
            elif not merged[0].isnumeric():
                None
            else:
                data.append(merged)
    return  cur_year,data
     

def overwrite_output(data):
    # ghi lại file output bằng data đã xử lý lần 1
    with open('output.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(data)
        
def separate_school_days(listed):
    # trong một môn học sẽ có nhiều tuần học khác nhau, nên cần tách các tuần ra
    listt= []
    listt +=     [listed[0]]

    for i in range(1,len(listed)-1):

        if len(listed[i]) ==11:
            if listed[i][9] != '' :
                index = listed[i][9]
                index_split = index.split(",")
                for j in index_split:
                    copyy = listed[i].copy()
                    copyy[9] = j 
                    listt +=[copyy]

        elif listed[i][8] != '' :

            index = listed[i][8]
            index_split = index.split(",")
            for j in index_split:
                copyy = listed[i].copy()
                copyy[8] = j 
                listt +=[copyy]
    return listt



# def convert_date(week):

#     day = 7*(int(week) -1)
#     date_start_2023 = '09/04/2023'
#     date_1 = datetime.datetime.strptime(date_start_2023 , '%m/%d/%Y').date()
#     event_date = date_1 + datetime.timedelta(days=day)
#     return event_date


def convert_number_day_to_day(week):
    # lấy cur_year đã xử lsy từ trước để xem năm học hiện đang xử lý và trả kết quả(date_start) ngày bắt đầu năm học
    if 'Năm học: 2021-2022' in cur_year:
        date_start = '06/09/2021'
    elif 'Năm học: 2022-2023' in cur_year:
        date_start = '12/12/2022'
    elif 'Năm học: 2023-2024' in cur_year:
        date_start = '04/09/2023'
    elif 'Năm học: 2024-2025' in cur_year:
        date_start = '04/09/2023'
    else :
        date_start = '04/09/2023'

    # vì trong cột "Tuần" chỉ hiện số tuần nên cần chuyển về số ngày và cộng với ngày bắt đầu năm học đó 
    day = 7*(int(week) -1)
    date_1 = datetime.datetime.strptime(date_start , '%m/%d/%Y').date()
    event_date = date_1 + datetime.timedelta(days=day)
    return event_date


            
# def convert_number_day(week):
#     day = 7*(int(week) -1)
#     return day

def convert_day(date):
    #Chuyển các thứ trong tuần thành số để cộng với tuần hiện tại để ra được ngày của tiết học
    if date == "Hai":
        return 0
    elif date == "Ba":
        return 1
    elif date == "Tư":
        return 2
    elif date == "Năm":
        return 3
    elif date == "Sáu":
        return 4
    elif date == "Bảy":
        return 5
    elif date == "Chủ Nhật":
        return 6

def schedule(lesson):
    #chuyển các tiết học thành dạng giờ bắt đầu tiết học
    if lesson == "1 - 3":
        return "07:00 AM"
    elif lesson == "4 - 6":
        return "09:30 AM"
    elif lesson == "7 - 9":
        return "01:00 PM"
    elif lesson == "10 - 12":
        return "03:30 PM"
    elif lesson == "13 - 15":
        return "06:00 PM"
def merge_description(data):
    # tạo "Description" để làm phần ghi chú trong gg calendar
        if "CBGD" in data.columns:
            data["Description"] = data["Mã lớp học phần"] +" "+data["CBGD"]
        else: data["Description"] = data["Mã lớp học phần"]

def add_col_private(data):
    # tạo "Private" để làm phần bảo mật trong gg calendar
    data["Private"]= False
def add_end_date(data):
    # tạo "End Date" để làm ngày kết thúc môn học trong gg calendar
    data["End Date"] = data["Tuần"]
def sort_column(data):
    #SẮP XẾP các cột để gg có thể đọc được file csv
    data = data[["Subject","Start Date","Start Time","End Date","End Time","Description","Location","Private"]]
def add_endtime(data):
    # tạo "End Time" để làm giờ kết thúc môn học trong gg calendar
    data["End Time"] = data["Start Time"].copy()

    #ghi lại "End Time"
    for i in range(len(data["End Time"])):
        if data["Start Time"][i] == "07:00 AM":
            data["End Time"][i] =  "09:30 AM"
        elif data["Start Time"][i] == "09:30 AM":
            data["End Time"][i] =  "12:00 AM"
        elif data["Start Time"][i] == "01:00 PM":
            data["End Time"][i] =  "03:30 PM"
        elif data["Start Time"][i] == "03:30 PM":
            data["End Time"][i] =  "06:00 PM"
        elif data["Start Time"][i] == "06:00 PM":
            data["End Time"][i] =  "08:30 PM"



def pdf_to_csv(inp):
    # inp = (r"")
    global cur_year #lấy cur_year
    oup = (r"output.csv")
    convert_into(input_path=inp,output_path = oup,output_format ="csv",pages = "all",lattice =True)
    cur_year,data_phase1=merge_relate_row('output.csv')
    data_phase2 =separate_school_days(data_phase1)
    data_phase3 = overwrite_output(data_phase2)
def csv_format_import_ggcalendar():
    usecols=["Mã lớp học phần","Tên học phần","Thứ","Phòng","CBGD","Tuần","Tiết"]
    data = pd.read_csv("output.csv",encoding="utf-8",usecols=lambda c: c in set(usecols),converters={"Thứ":convert_day,'Tuần':convert_number_day_to_day,'Tiết':schedule})
    for i in range(len(data)):
        day = data["Thứ"][i]
        if day== int:
            data['Tuần'][i] += datetime.timedelta(days=int(day))
    merge_description(data)
    add_col_private(data)
    add_end_date(data)
    data.rename(columns={"Tên học phần":"Subject","Tiết":"Start Time","Tuần":'Start Date',"Phòng":"Location"},inplace=True)
    add_endtime(data)
    sort_column(data)
    dataed = data[["Subject","Start Date","Start Time","End Date","End Time","Description","Location","Private"]].copy()
    dataed.to_csv("output.csv", sep=',', index=False, encoding='utf-8')


input_pdf = input("File pdf path: ")
pdf_to_csv(input_pdf)
csv_format_import_ggcalendar